In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import os
import sys
import traceback

print("Starting train.py...")

class LandmarkEmotionTransformer(nn.Module):
    def __init__(self, num_landmarks=468, num_emotions=6, d_model=128, nhead=8, num_layers=4):
        super(LandmarkEmotionTransformer, self).__init__()
        self.num_landmarks = num_landmarks
        self.d_model = d_model
        self.num_emotions = num_emotions

        self.input_embedding = nn.Linear(3, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(num_landmarks * d_model, num_emotions)

    def forward(self, landmarks):
        batch_size = landmarks.size(0)
        x = self.input_embedding(landmarks)  # (batch, 468, d_model)
        x = self.transformer_encoder(x)      # (batch, 468, d_model)
        x = x.reshape(batch_size, -1)        # (batch, 468 * d_model)
        logits = self.fc(x)                  # (batch, num_emotions)
        return logits

class EmotionDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def load_data(file_path):
    print(f"Loading dataset from: {file_path}")
    try:
        df = pd.read_csv(file_path)
        print(f"Raw dataset shape: {df.shape}")
        
        df = df.dropna(subset=['Expression'])
        print(f"Shape after removing NaN labels: {df.shape}")
        
        # Diagnose emotions
        unique_emotions = df['Expression'].unique()
        print(f"Unique emotions in dataset: {unique_emotions}")
        if len(unique_emotions) == 0:
            raise ValueError("No valid emotions found")
        
        # Normalize emotions
        df['Expression'] = df['Expression'].str.strip().str.lower()
        unique_emotions = df['Expression'].unique()
        print(f"Normalized unique emotions: {unique_emotions}")
        
        feature_cols = [col for col in df.columns if col != 'Expression']
        expected_features = 468 * 3
        if len(feature_cols) != expected_features:
            raise ValueError(f"Expected {expected_features} feature columns, got {len(feature_cols)}")
        
        features = df[feature_cols].values
        if np.any(np.isnan(features)) or np.any(np.isinf(features)):
            print("Warning: Found NaN or infinite values in features. Imputing with mean...")
            features = np.nan_to_num(features, nan=np.nanmean(features, axis=0), posinf=np.nanmean(features, axis=0), neginf=np.nanmean(features, axis=0))
        
        scaler = StandardScaler()
        features = scaler.fit_transform(features)
        features = features.reshape(-1, 468, 3)
        
        label_encoder = LabelEncoder()
        labels = label_encoder.fit_transform(df['Expression'])
        
        print(f"Label mapping: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")
        print(f"Feature shape after reshape: {features.shape}")
        
        return features, labels, label_encoder, scaler
    except Exception as e:
        print(f"Error in load_data: {e}")
        traceback.print_exc()
        sys.exit(1)



Starting train.py...


In [ ]:

def train_model():
    print("Entering train_model...")
    num_landmarks = 468
    d_model = 128
    nhead = 8
    num_layers = 4
    batch_size = 32
    epochs = 50
    learning_rate = 0.0001

    dataset_path = 'JoyVerseDataSet_Filled.csv'
    print(f"Resolved dataset path: {os.path.abspath(dataset_path)}")
    features, labels, label_encoder, scaler = load_data(dataset_path)

    num_emotions = len(label_encoder.classes_)
    print(f"Number of emotions: {num_emotions}")

    print("Splitting data...")
    X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.2, random_state=42
    )

    print("Creating datasets...")
    train_dataset = EmotionDataset(X_train, y_train)
    test_dataset = EmotionDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    print("Initializing model...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    model = LandmarkEmotionTransformer(
        num_landmarks=num_landmarks,
        num_emotions=num_emotions,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers
    ).to(device)

    print("Setting up optimizer and loss...")
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    print("Starting training loop...")
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        for batch_features, batch_labels in train_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            if torch.isnan(loss):
                print(f"Warning: NaN loss at epoch {epoch+1}. Stopping training.")
                return
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
        
        train_acc = 100 * train_correct / train_total
        train_loss_avg = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch_features, batch_labels in test_loader:
                batch_features = batch_features.to(device)
                batch_labels = batch_labels.to(device)
                outputs = model(batch_features)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()
        val_acc = 100 * val_correct / val_total
        val_loss_avg = val_loss / len(test_loader)

        print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss_avg:.4f} | Train Accuracy: {train_acc:.2f}% | Val Loss: {val_loss_avg:.4f} | Val Accuracy: {val_acc:.2f}%')

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'emotion_detector.pth')
            joblib.dump(scaler, 'scaler.pkl')
            print(f"Saved best model with validation accuracy: {best_acc:.2f}%")

    print(f"Training completed. Best validation accuracy: {best_acc:.2f}%")
    print("Saving label encoder...")
    np.save('label_encoder.npy', label_encoder.classes_)


In [ ]:
if __name__ == '__main__':
    try:
        print("Executing main block...")
        train_model()
    except Exception as e:
        print(f"Error in main: {e}")
        traceback.print_exc()
        sys.exit(1)